# Crawl D&C places from the Church of Jesus Christ of Latter-day Saints

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup

from models.crawl_utils import get_page, save_page

In [4]:
# config
base = 'https://doctrineandcovenantscentral.org'
host = 'https://doctrineandcovenantscentral.org/places-of-the-doctrine-and-covenants/'
base_dir = '../data/raw/dc_places'
bs_parser = 'html.parser'
delay_seconds = 30

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    

In [22]:
def extract_hrefs(html):
    hrefs = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div tags with the class 'views-field-title'
    link_tags = soup.find_all('div', {'data-column-clickable': True})
    print(link_tags)

    # Extract href attribute from each anchor tag and append to the list
    for link in link_tags:
        href = link.get('data-column-clickable')
        if href:
            href = base + href
            hrefs.append(href)

    return hrefs

In [20]:
def get_path(url):
    path_components = urlparse(url).path.split('/')
    print(path_components)
    return os.path.join(base_dir, f"{path_components[-2]}.json")

In [25]:
dc_places_links = []

status_code, html = get_page(host, delay_seconds)
if status_code != 200:
    print("Error!", status_code)
hrefs = extract_hrefs(html)
print(hrefs)
if len(hrefs) != 0:
    dc_places_links.extend(hrefs)
    
print(dc_places_links)
len(dc_places_links)

[<div class="has_eae_slider make-column-clickable-elementor elementor-column elementor-col-33 elementor-inner-column elementor-element elementor-element-340b05c" data-column-clickable="/palmyra-new-york/" data-column-clickable-blank="_self" data-element_type="column" data-id="340b05c" style="cursor: pointer;">
<div class="elementor-column-wrap elementor-element-populated">
<div class="elementor-widget-wrap">
<div class="elementor-element elementor-element-c1147e9 elementor-widget elementor-widget-image" data-element_type="widget" data-id="c1147e9" data-widget_type="image.default">
<div class="elementor-widget-container">
<div class="elementor-image">
<img alt="" class="elementor-animation-float attachment-large size-large" height="350" loading="lazy" sizes="100vw" src="https://doctrineandcovenantscentral.org/wp-content/uploads/2021/02/Palmyra-1024x682.png" srcset="https://doctrineandcovenantscentral.org/wp-content/uploads/2021/02/Palmyra-1024x682.png 1024w, https://doctrineandcovenants

16

In [26]:
for url in dc_places_links:
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    status_code, html = get_page(url, delay_seconds)
    if status_code != 200:
        print("Error!", status_code , url)
        continue
    save_page(path_file,url,html)
    
print("End")

['', 'palmyra-new-york', '']
../data/raw/dc_places/palmyra-new-york.json
['', 'harmony-pennsylvania', '']
../data/raw/dc_places/harmony-pennsylvania.json
['', 'fayette-new-york', '']
../data/raw/dc_places/fayette-new-york.json
['', 'perrysburg-new-york', '']
../data/raw/dc_places/perrysburg-new-york.json
['', 'kirtland-ohio', '']
../data/raw/dc_places/kirtland-ohio.json
['', 'hiram-ohio', '']
../data/raw/dc_places/hiram-ohio.json


ConnectTimeout: HTTPSConnectionPool(host='doctrineandcovenantscentral.org', port=443): Max retries exceeded with url: /hiram-ohio/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fa1e64acee0>, 'Connection to doctrineandcovenantscentral.org timed out. (connect timeout=30)'))